creación del directorio para instalar el modelo

In [ ]:
!mkdir src

Traer el modelo, se llama final_model y es la versión 1

In [ ]:
model_name = 'final_model'
model_version = 1
model = mlflow.pyfunc.load_model(f"models:/{model_name}/{model_version}")
display(model)

creación del script para la ejecución del modelo

In [ ]:
import shutil
origen = "/content/dataset/102 flower/cat_to_name.json"
destino = "/content/src"
shutil.copy(origen, destino)

In [ ]:
%%writefile ./src/final_model.py
from argparse import ArgumentParser
import mlflow
import tensorflow as tf
import numpy as np
import json
import os

mlflow.set_tracking_uri("http://localhost:5000")
model_name = 'final_model'
model_version = 1
model = mlflow.pyfunc.load_model(f"models:/{model_name}/{model_version}")
c = [41, 43, 46, 51, 58, 73, 74, 75, 77, 78, 80, 81, 82, 83, 88, 89, 94, 95]
ordered_to_id = {i: j for i, j in enumerate(c)}
id_to_ordered = {j: i for i, j in ordered_to_id.items()}
#path_cat = os.path.join(os.path.dirname(__file__), "cat_to_name.json")
path_cat = "/content/src/cat_to_name.json"
with open(path_cat) as r:
  dict_flower = json.load(r)
dict_flower = {int(key): value for key, value in dict_flower.items()}


def preprocess_image(path_image):
  img = tf.keras.preprocessing.image.load_img(
      path_image, target_size = (224, 224, 3)
  )
  img = np.array([img])
  return img

def classify_image(path_image):
  img = preprocess_image(path_image)
  prediction = model.predict(img).argmax(axis=1)
  return prediction

def convert_id(value, via):
  if via == 'to_id':
    return ordered_to_id[value]
  if via == 'to_ordered':
    return id_to_ordered[value]

def correct_classify(prediction):
  correct_prediction = convert_id(prediction, "to_id")
  pred_fl = dict_flower.get(correct_prediction, "Flor no identificada")
  return pred_fl

def main():
  parser = ArgumentParser(
      description = "CLI para modelo de clasificación de flores"
  )
  parser.add_argument(
      "--text", type=str, required=True, help="Ruta de la imagen a clasificar"
  )
  args = parser.parse_args()
  prediction = classify_image(args.text)[0]
  correct_pred = correct_classify(prediction)
  print(f"El tipo de flor en la imagen es: {correct_pred}")

if __name__ == "__main__":
  main()

In [ ]:
!python ./src/final_model.py --help

In [ ]:
!python ./src/final_model.py --text "dataset/102 flower/flowers/test/51/image_01314.jpg"

crearemos el cli para poder instalar el script en el sistema

In [ ]:
%%writefile ./src/pyproject.toml
[project]
name = "mlcli"
version = "0.1.0"
description = "CLI para la detección de tipo de flor por medio de imagen"
authors = [
  {name="Diana Carolina Diaz Ortigoza", email="dicdiazor@unal.edu.co"},
  {name="Julio Cesar Roa Davila", email="jucroada@unal.edu.co"},
  {name="Jair Sebastian Saavedra Alvarado", email="jssaavedraa@unal.edu.co"},
]
dependencies = [
  "argparse >= 1.1",
  "mlflow >= 2.13.0",
  "tensorflow == 2.15.0",
  "numpy >= 1.25.2"
]

[build-system]
requires = [
  "setuptools", "wheel"
]

[project.scripts]
pred_model = "final_model:main"

In [ ]:
!pip install ./src/

In [ ]:
!pred_model --text "dataset/102 flower/flowers/test/51/image_01314.jpg"

2024-05-29 04:53:20.247221: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-29 04:53:20.247311: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-29 04:53:20.249481: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-29 04:53:22.024897: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
1/1 [==============================] - 0s 327ms/step
El tipo de flor en la imagen es: frangipani
